In [1]:
%%capture
# above used to suppress output when calling onnx_to_keras

import onnx
from onnx2keras import onnx_to_keras

datasets = ['CIFAR', 'MNIST']
nets = ['ConvBig', 'ConvMed', 'FFNN']

models = {}

for dataset in datasets:
    models[dataset] = {}
    for net in nets:
        onnx_model = onnx.load(f'networks_onnx/{dataset}_Networks/{net}.onnx')
        
        if net == 'FFNN':
            k_model = onnx_to_keras(onnx_model, ['0'])
        else:
            k_model = onnx_to_keras(onnx_model, ['input.1'])
        
        models[dataset][net] = k_model

In [2]:
convmed_layers = [2, 5, 8, 10]
convbig_layers = [2, 5, 8, 11, 14, 16, 18]
ffnn_layers = [2, 4, 6, 8, 10, 12, 14]

from keras import backend as K

In [3]:
for dataset in datasets:
    for net in nets:
        model = models[dataset][net]
        
        if net == 'ConvBig':
            layers = convbig_layers
        elif net == 'ConvMed':
            layers = convmed_layers
        else:
            layers = ffnn_layers
        
        n = 1
        for layer_i in layers:
            w = K.constant(model.layers[layer_i].get_weights()[0])
            
            axes = 0
            if len(w.shape) == 4:
                axes=[0, 1, 2]
            
            norm = K.max(K.sum(K.abs(w), axis=axes, keepdims=False))
            n *= norm
            
        print(f'{dataset} {net}: {n}')

CIFAR ConvBig: 1587185536.0
CIFAR ConvMed: 64603.76171875
CIFAR FFNN: 97618464.0
MNIST ConvBig: 3638838016.0
MNIST ConvMed: 92872.0859375
MNIST FFNN: 165186992.0
